# 3.1 Token 与分词可视化

本笔记本将帮助你理解 LLM 的基本单位 —— Token，以及它如何影响 API 成本。

**成本**: $0 (仅使用 tiktoken 库，无需 API 调用)

**安装依赖**:
```bash
pip install tiktoken
```

In [ ]:
# Install dependencies
!pip install tiktoken -q

In [ ]:
import tiktoken
from typing import List

def visualize_tokens(text: str, encoding_name: str = "cl100k_base") -> None:
    """可视化文本的分词结果"""
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(text)
    
    print(f"原文: {text}")
    print(f"\nToken 数量: {len(tokens)}")
    print(f"\nToken IDs: {tokens}")
    print(f"\n分词结果:")
    
    for i, token_id in enumerate(tokens):
        token_str = encoding.decode([token_id])
        print(f"  [{i}] ID={token_id:6d} → '{token_str}'")

## Demo 1: 英文 vs 中文分词Comparison
<!-- Demo 1: 英文 vs 中文分词对比 -->

观察英文和中文在分词上的差异。通常，中文每个汉字会占用更多的 token。

In [ ]:
# 英文文本
english_text = "Hello, how are you today?"
print("=" * 60)
print("英文分词:")
print("=" * 60)
visualize_tokens(english_text)

print("\n" + "=" * 60)

# 中文文本(大致相同含义)
chinese_text = "你好,你今天怎么样?"
print("中文分词:")
print("=" * 60)
visualize_tokens(chinese_text)

## Demo 2: 计算相同句子的Cost差异
<!-- Demo 2: 计算相同句子的成本差异 -->

比较英文和中文版本相同含义句子的 token 数量和成本。

In [ ]:
def estimate_cost(text: str, encoding_name: str = "cl100k_base",
                  input_price_per_1m: float = 2.50,
                  output_price_per_1m: float = 10.00) -> dict:
    """估算 API 调用成本
    
    Args:
        text: 输入文本
        encoding_name: 编码器名称
        input_price_per_1m: 每百万输入 token 的价格(美元)
        output_price_per_1m: 每百万输出 token 的价格(美元)
    
    Returns:
        包含 token 数量和成本的字典
    """
    encoding = tiktoken.get_encoding(encoding_name)
    token_count = len(encoding.encode(text))
    
    input_cost = (token_count / 1_000_000) * input_price_per_1m
    output_cost = (token_count / 1_000_000) * output_price_per_1m
    
    return {
        "token_count": token_count,
        "input_cost_usd": input_cost,
        "output_cost_usd": output_cost,
        "total_cost_usd": input_cost + output_cost
    }

# 示例句子
english_sentence = "Artificial intelligence is transforming how we build software applications."
chinese_sentence = "人工智能正在改变我们构建软件应用程序的方式。"

print("GPT-4o 定价 (2026年2月):")
print("  输入: $2.50 / 1M tokens")
print("  输出: $10.00 / 1M tokens")
print("\n" + "=" * 60)

en_cost = estimate_cost(english_sentence)
print(f"\n英文: '{english_sentence}'")
print(f"  Token 数量: {en_cost['token_count']}")
print(f"  输入成本: ${en_cost['input_cost_usd']:.6f}")
print(f"  输出成本: ${en_cost['output_cost_usd']:.6f}")

print("\n" + "=" * 60)

cn_cost = estimate_cost(chinese_sentence)
print(f"\n中文: '{chinese_sentence}'")
print(f"  Token 数量: {cn_cost['token_count']}")
print(f"  输入成本: ${cn_cost['input_cost_usd']:.6f}")
print(f"  输出成本: ${cn_cost['output_cost_usd']:.6f}")

print("\n" + "=" * 60)
print(f"\n中文 Token 数量是英文的 {cn_cost['token_count'] / en_cost['token_count']:.2f} 倍")
print(f"中文成本是英文的 {cn_cost['total_cost_usd'] / en_cost['total_cost_usd']:.2f} 倍")

## Demo 3: 不同 Tokenizer 编码Comparison
<!-- Demo 3: 不同 Tokenizer 编码对比 -->

不同的模型使用不同的 tokenizer:
- **cl100k_base**: GPT-4, GPT-3.5-turbo
- **o200k_base**: GPT-4o 系列(更高效,特别是对非英语文本)

In [ ]:
test_text = "人工智能 (AI) 正在改变世界! Artificial Intelligence is transforming the world!"

encodings = [
    ("cl100k_base", "GPT-4, GPT-3.5-turbo"),
    ("o200k_base", "GPT-4o 系列")
]

print(f"测试文本: '{test_text}'\n")
print("=" * 60)

results = []
for encoding_name, models in encodings:
    encoding = tiktoken.get_encoding(encoding_name)
    tokens = encoding.encode(test_text)
    token_count = len(tokens)
    results.append((encoding_name, models, token_count))
    
    print(f"\n{encoding_name} ({models}):")
    print(f"  Token 数量: {token_count}")
    print(f"  Token IDs: {tokens[:10]}{'...' if len(tokens) > 10 else ''}")

print("\n" + "=" * 60)
print("\n总结:")
baseline = results[0][2]
for encoding_name, models, count in results:
    efficiency = (1 - count/baseline) * 100 if count < baseline else 0
    print(f"  {encoding_name}: {count} tokens", end="")
    if efficiency > 0:
        print(f" (节省 {efficiency:.1f}%)")
    else:
        print()

## Demo 4: 估算 API Cost
<!-- Demo 4: 估算 API 成本 -->

针对一段实际文本,估算使用不同模型的成本。

In [ ]:
# 一段技术文档示例
document = """
大语言模型(Large Language Model, LLM)是一种基于深度学习的自然语言处理模型。
它通过在海量文本数据上进行预训练,学习语言的统计规律和语义关系。
GPT-4o 是 OpenAI 推出的多模态模型,支持文本、图像和音频输入。
相比前代模型,GPT-4o 在处理速度和成本效率上都有显著提升。
它特别优化了对中文、日文等非英语语言的支持,token 效率提升约 50%。

在实际应用中,选择合适的模型需要权衡:
1. 任务复杂度 - 简单任务可使用 GPT-3.5 或 GPT-4o-mini
2. 成本预算 - GPT-4o-mini 比 GPT-4o 便宜约 10 倍
3. 响应速度 - 较小模型通常响应更快
4. 输出质量 - 复杂推理任务需要更强大的模型
"""

# 定义不同模型的定价(2026年2月,美元/百万tokens)
models = [
    ("GPT-4o", "o200k_base", 2.50, 10.00),
    ("GPT-4o-mini", "o200k_base", 0.15, 0.60),
    ("GPT-4-turbo", "cl100k_base", 10.00, 30.00),
    ("GPT-3.5-turbo", "cl100k_base", 0.50, 1.50),
]

print(f"文档长度: {len(document)} 字符\n")
print("=" * 80)
print(f"{'模型':<18} {'Token':<8} {'输入成本':<12} {'输出成本':<12} {'总成本':<12}")
print("=" * 80)

for model_name, encoding_name, input_price, output_price in models:
    encoding = tiktoken.get_encoding(encoding_name)
    token_count = len(encoding.encode(document))
    
    input_cost = (token_count / 1_000_000) * input_price
    output_cost = (token_count / 1_000_000) * output_price  # 假设输出与输入等长
    total_cost = input_cost + output_cost
    
    print(f"{model_name:<18} {token_count:<8} ${input_cost:<11.6f} ${output_cost:<11.6f} ${total_cost:<11.6f}")

print("=" * 80)
print("\n注: 以上成本假设输出长度与输入相同。实际成本取决于具体的输出长度。")

## Demo 5: 上下文窗口限制

每个模型都有最大 token 限制(上下文窗口)。超过限制会导致 API 调用失败。

In [ ]:
def check_context_window(text: str, model: str, max_tokens: int, 
                         encoding_name: str = "o200k_base") -> dict:
    """检查文本是否超出模型的上下文窗口"""
    encoding = tiktoken.get_encoding(encoding_name)
    token_count = len(encoding.encode(text))
    
    remaining = max_tokens - token_count
    usage_percent = (token_count / max_tokens) * 100
    
    return {
        "model": model,
        "token_count": token_count,
        "max_tokens": max_tokens,
        "remaining": remaining,
        "usage_percent": usage_percent,
        "is_valid": token_count <= max_tokens
    }

# Generate一段长文本
long_text = "这是一段测试文本。" * 1000  # 重复1000次

# 不同模型的上下文窗口限制
model_limits = [
    ("GPT-4o", 128000, "o200k_base"),
    ("GPT-4o-mini", 128000, "o200k_base"),
    ("GPT-4-turbo", 128000, "cl100k_base"),
    ("GPT-3.5-turbo", 16385, "cl100k_base"),
]

print(f"测试文本长度: {len(long_text)} 字符\n")
print("=" * 80)
print(f"{'模型':<18} {'Token 数':<12} {'上限':<12} {'剩余':<12} {'使用率':<12} {'状态'}")
print("=" * 80)

for model, max_tokens, encoding_name in model_limits:
    result = check_context_window(long_text, model, max_tokens, encoding_name)
    
    status = "✓ 可用" if result["is_valid"] else "✗ 超限"
    print(f"{result['model']:<18} "
          f"{result['token_count']:<12} "
          f"{result['max_tokens']:<12} "
          f"{result['remaining']:<12} "
          f"{result['usage_percent']:<11.1f}% "
          f"{status}")

print("=" * 80)
print("\n建议: 对于长文档处理,预留至少 20-30% 的空间用于系统提示和输出。")

## 练习: Analysis你自己的文本
<!-- 练习: 分析你自己的文本 -->

将你工作中的一段文本粘贴到下面,分析它的 token 数量和 API 成本。

In [ ]:
# 在这里粘贴你的文本
your_text = """
在这里粘贴你想分析的文本...
"""

if your_text.strip() and your_text.strip() != "在这里粘贴你想分析的文本...":
    print("文本分析结果:\n")
    print("=" * 80)
    
    # 分词可视化(只显示前20个token)
    encoding = tiktoken.get_encoding("o200k_base")
    tokens = encoding.encode(your_text)
    print(f"总 Token 数: {len(tokens)}\n")
    print("前 20 个 Tokens:")
    for i, token_id in enumerate(tokens[:20]):
        token_str = encoding.decode([token_id])
        print(f"  [{i}] '{token_str}'")
    if len(tokens) > 20:
        print(f"  ... 还有 {len(tokens) - 20} 个 tokens")
    
    print("\n" + "=" * 80)
    print("\n成本估算 (假设输出与输入等长):\n")
    
    # 成本估算
    models_for_estimate = [
        ("GPT-4o", "o200k_base", 2.50, 10.00),
        ("GPT-4o-mini", "o200k_base", 0.15, 0.60),
    ]
    
    for model_name, encoding_name, input_price, output_price in models_for_estimate:
        encoding = tiktoken.get_encoding(encoding_name)
        token_count = len(encoding.encode(your_text))
        
        input_cost = (token_count / 1_000_000) * input_price
        output_cost = (token_count / 1_000_000) * output_price
        total_cost = input_cost + output_cost
        
        print(f"{model_name}:")
        print(f"  Token 数: {token_count}")
        print(f"  输入成本: ${input_cost:.6f}")
        print(f"  输出成本: ${output_cost:.6f}")
        print(f"  总成本: ${total_cost:.6f}")
        print()
else:
    print("请在上面的 your_text 变量中粘贴你的文本,然后重新运行此单元格。")

## 总结

通过本笔记本,你应该了解到:

1. **Token 是什么**: LLM 处理文本的基本单位,通常一个 token 约等于 4 个英文字符或 1.5 个中文字符

2. **语言差异**: 中文通常比英文消耗更多 token,但 GPT-4o 的新 tokenizer (o200k_base) 显著改善了这一问题

3. **成本计算**: API 成本直接与 token 数量相关,输入和输出分别计费

4. **模型选择**: 根据任务复杂度和预算选择合适的模型

5. **上下文限制**: 注意每个模型的最大 token 限制,为输出预留足够空间

**最佳实践**:
- 使用 tiktoken 在发送请求前预估成本
- 对于中文应用,优先选择 GPT-4o 系列(使用 o200k_base tokenizer)
- 简单任务使用 mini 模型节省成本
- 监控 token 使用量,避免超出上下文窗口